In [39]:
# Basic Imports
import pandas as pd, numpy as np

# Warning Imports
import warnings     
warnings.filterwarnings("ignore")

# Scikit-learn Imports
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Other Imports
import spacy
import re

# Gensim Imports
from gensim.models import Word2Vec

# NLTK Imports
import nltk
from nltk.stem import PorterStemmer       # For Stemming
from nltk.stem import WordNetLemmatizer   # For Lemmatization
from nltk.corpus import stopwords

In [37]:
# Defining the objects
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()



In [24]:
stopwords = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

In [25]:
# https://www.kaggle.com/datasets/kazanova/sentiment140
# -----------------------------------------------------
file_path = r"../Data/tweet_sentiment.csv"
df = pd.read_csv(file_path, delimiter=',', encoding='latin-1', header=None)
df.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
display(df.shape)
display(df.info())
display(df.head())

(1600000, 6)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column  Non-Null Count    Dtype 
---  ------  --------------    ----- 
 0   target  1600000 non-null  int64 
 1   ids     1600000 non-null  int64 
 2   date    1600000 non-null  object
 3   flag    1600000 non-null  object
 4   user    1600000 non-null  object
 5   text    1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB


None

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [26]:
# Stratified Sampling
# -------------------
df1, _ = sklearn.model_selection.train_test_split(df,test_size=0.9, stratify=df['target'])
df1.shape

(160000, 6)

-----------------

### <font color = 'red'> **Tokenize** </font>

-----------------

In [31]:

df1['tokenized_text'] = df1['text'].apply(nltk.sent_tokenize)
df1.head()

,target,ids,date,flag,user,text,tokenized_text
738780,0,2265557168,Sun Jun 21 06:50:33 PDT 2009,NO_QUERY,Shayzorz,@aka_maye wish i could be watching it with you...,[@aka_maye wish i could be watching it with yo...
1376608,4,2051774046,Fri Jun 05 22:46:38 PDT 2009,NO_QUERY,MzAmezcua02,MY NAME ISNT WILLIS. STUPID ASS BOBO LOL TEE--...,"[MY NAME ISNT WILLIS., STUPID ASS BOBO LOL TEE..."
36211,0,1565577957,Mon Apr 20 07:05:02 PDT 2009,NO_QUERY,alealex26,"sacrifice, fulfillment and gratitude..my heart...","[sacrifice, fulfillment and gratitude..my hear..."
1499800,4,2070973600,Sun Jun 07 18:36:59 PDT 2009,NO_QUERY,xobreanna,being amused at heidi pratt being a christian ...,[being amused at heidi pratt being a christian...
211766,0,1974571090,Sat May 30 13:24:20 PDT 2009,NO_QUERY,frost914,"Last Jay Leno episode was good, can't wait for...","[Last Jay Leno episode was good, can't wait fo..."


In [34]:
def pre_processing_1(sentence):
    corpus = []
    for i in range(len(sentence)):
        review = re.sub('[^a-zA-Z]',' ', sentence[i])
        review = review.lower()
        review = review.split()
        review = [lemmatizer.lemmatize(word) for word in review if not word in set(stopwords)]
        review = " ".join(review)
        corpus.append(review)
    return corpus

df1['preprocessed_1'] = df1['tokenized_text'].apply(pre_processing_1)   
df1.head()

In [40]:
s1 = [['word', 'embeddings', 'are', 'useful'],
             ['word', 'embeddings', 'capture', 'semantic', 'relationships'],
             ['gensim', 'is', 'a', 'library', 'for', 'Word2Vec']]
model = Word2Vec(s1, vector_size=100, window=5, min_count=1, workers=4)
word_embedding = model.wv['word']
print(word_embedding[0:10])